In [1]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

# update batch size
batch_size = 20

# number of batch in training
num_of_batch = 10

# fixed number of time steps in one episode (not used)
trading_period = 60

# 1 is zscore
num_features = 1

# 0 is no position. 1 is long the spread. 2 is short the spread.
position_num = 3

# RNN hidden state dimension
h_dim = 70

# number of RNN layer
num_layers = 1

# number of actions
a_num = 4

# number of layer1 output
layer1_out_num = 100

# learning rate
lr = 1e-3

In [2]:
class TradingPolicyModel(tf.keras.Model):
    def __init__(self):
        super(TradingPolicyModel, self).__init__()
        self.dense1 = tf.layers.Dense(units=layer1_out_num,
                                      activation=tf.keras.layers.LeakyReLU(),
                                      kernel_initializer=tf.contrib.layers.xavier_initializer()
                                     )
        self.logits = tf.layers.Dense(units=a_num,
                                      activation=tf.keras.layers.LeakyReLU(),
                                      kernel_initializer=tf.contrib.layers.xavier_initializer()
                                     )

    def call(self, inputs):
        # Forward pass
        x = self.dense1(inputs)
        logits = self.logits(x)
        return logits

def sample_action(logits):
    dist = tf.distributions.Categorical(logits=logits)
    
    # 1-D Tensor where the i-th element correspond to a sample from
    # the i-th categorical distribution
    return dist.sample().numpy()


class StateEncodingModel(tf.keras.Model):
    def __init__(self):
        super(TradingPolicyModel, self).__init__()
        self.cell_layer = tf.contrib.rnn.LSTMCell(h_dim)
        sef.cell = tf.contrib.rnn.MultiRNNCell([self.cell_layer] * num_layers)
        self.state = cell.zero_state(batch_size, tf.float32)
    
    def call(self, inputs):
        oberservation, self.state = cell(inputs, self.state)
        return oberservation
        
    def reset_state(self):
        self.state = cell.zero_state(batch_size, tf.float32)


def get_random_history(history):
    """Sample some pairs and get the history of those pairs. The history should have
    three dimension. The first dimension is for time. The second dimension is indexed
    by features name. The third dimension is the index of training instance.
    """
    pass

def compute_input_history(history):
    """Slicing history in its second dimension."""
    pass

class TradingEnvironment():
    """Trading environment for reinforcement learning training.
    
    Arguments:
        state_encoding_model: the model that encode past input_history data into a state
        vector which will be fed as input to the policy network.
    """
    def __init__(self, state_encoding_model):
        # do some initialization
        self.state_encoding_model = state_encoding_model
        self._reset_env()
        
    def _reset_env(self):
        self.t = 0
        self.rnn_state = self.state_encoding_model.zero_state()

        # 0 is no position. 1 is long the spread. 2 is short the spread
        self.position = np.zeros(batch_size)

        # prepare a batch of history and input_history
        self.history = get_random_history(batch_size)
        self.input_history = compute_input_history(self.history)
        
        # create or update self.state variable
        self.update_state()
        
    
    def reset(self):
        """Return an initial state for the trading environment"""
        if self.t == 0:
            return self.state
        else:
            self._reset_env()
            return self.state
    
    def reward_magic_func(self):
        pass
    
    def update_state(self):
        # concate next_input_history and next position to form next partial state
        partial_state = tf.concat([self.input_history[self.t], tf.one_hot(self.position, position_num)], 1)
        
        # update state
        self.state, self.rnn_state = self.state_encode_model(partial_state, self.rnn_state)
    
    def compute_next_state(self, action):
        # compute_next_position
        next_pos = np.zeros_like(self.position)
        next_pos += (action == 0)*self.position
        next_pos += (action == 1)*1
        next_pos += (action == 2)*2
        # this line has no effect as next_pos is initialized as zero
#         next_pos += (action == 3)*0
        self.position = next_pos
        
        self.update_state()
    
    def step(self, action):
        """Given the current state and action, return the reward, next state and done.
        This function should be called after reset.
        
        
        Arguments:
            action: a numpy array containing the current action for each training pair.

        Note that we follow the convention where the trajectory is indexed as s_0, a_0, r_0,
        s_1, ... . Therefore t is updated just after computing the reward is computed and
        before computing next state.
        """
        r = self.reward_magic_func()
        self.t += 1
        self.compute_next_state(action)
        return r, self.state, self.t == batch_size

def loss(all_logits, all_actions, all_advantages):
    neg_log_select_prob = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=all_logits, labels=all_actions)
    
    # 0 axis is the time axis. 1 axis is the batch axis
    return tf.reduce_mean(neg_log_select_prob * all_advantages, 0)

In [ ]:
pi = TradingPolicyModel()
state_encode_model = StateEncodingModel()
env = TradingEnvironment(state_encode_model)

for batch in range(num_of_batch):

    all_logits = []
    all_actions = []
    all_rewards = []
    with tf.GradientTape() as gt:
        done = False
        s = env.reset()

        # internally the episode length is fixed by trading_period
        while not done:
            logits = pi(0)
            a = sample_action(logits)
            r, next_s, done = senv.step(a.numpy())
        
            all_logits.append(logits)
            all_actions.append(a)
            all_rewards.append(r)
        
        loss(tf.stack(all_logits), tf.stack(all_actions), )
        